## Домашнее задание

1. Необходимо подготовить датасет https://www.kaggle.com/olekslu/makeup-lips-segmentation-28k-samples для обучения модели на сегментацию губ
2. Обучить модель на выбор из [segmentation_models_pytorch](https://segmentation-modelspytorch.readthedocs.io/en/latest/index.html)

In [29]:
!pip install segmentation-models-pytorch

import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import os
from segmentation_models_pytorch import Unet
import torch.nn.functional as F
from torch import optim

In [30]:
class LipDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform

        self.image_files = os.listdir(images_dir)
        self.mask_files = os.listdir(masks_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        mask_path = os.path.join(self.masks_dir, self.mask_files[idx])

        image = Image.open(image_path)
        mask = Image.open(mask_path).convert('L')  # изменение режима изображения на 'L'

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

In [31]:
# путь к папке с изображениями губ
images_dir = "/Users/annvorosh/Documents/GB/PyTorch/set-lipstick-original/720p"

# путь к папке с масками губ
masks_dir = "/Users/annvorosh/Documents/GB/PyTorch/set-lipstick-original/mask"

In [32]:
# размер изображений и масок
image_size = (256, 256)

# загрузка датасета и разделение на тренировочную и валидационную выборки
dataset = LipDataset(images_dir, masks_dir, transform=transforms.Compose([transforms.Resize(image_size),
                                                                           transforms.ToTensor()]))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# загрузка тренировочной и валидационной выборки в DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [33]:
# создание экземпляра модели UNet и определение функции потерь
model = Unet('resnet34', encoder_weights='imagenet', classes=1, activation=None)
criterion = F.binary_cross_entropy_with_logits

# определение оптимизатора и scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)

In [ ]:
# обучение модели
for epoch in range(2):
    train_loss = 0
    val_loss = 0

    # тренировочный этап
    model.train()
    for images, masks in train_loader:
        optimizer.zero_grad()

        output_masks = model(images)
        loss = criterion(output_masks, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # валидационный этап
    model.eval()
    with torch.no_grad():
        for images, masks in val_loader:
            output_masks = model(images)
            loss = criterion(output_masks, masks)
            val_loss += loss.item() * images.size(0)

    # выводим средний loss на каждой эпохе
    train_loss /= len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)
    print(f"Epoch {epoch+1}. Train loss: {train_loss:.4f}. Val loss: {val_loss:.4f}")

    # обновление scheduler
    scheduler.step(val_loss)

In [ ]:
# сохранение обученной модели
torch.save(model.state_dict(), "lip_segmentation_model.pth")